In [1]:
import requests

In [2]:
import pandas as pd

In [3]:
import time

In [4]:
from openpyxl import Workbook

In [5]:
from openpyxl.utils.dataframe import dataframe_to_rows

In [6]:
def fetch_crypto_data():
    url = "https://api.coingecko.com/api/v3/coins/markets"
    params = {
        "vs_currency": "usd",
        "order": "market_cap_desc",
        "per_page": 50,
        "page": 1,
        "sparkline": False
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    print(data)  # Print before returning
    return data
fetch_crypto_data()

[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 83784, 'market_cap': 1659945326387, 'market_cap_rank': 1, 'fully_diluted_valuation': 1659945326387, 'total_volume': 74109155624, 'high_24h': 93668, 'low_24h': 82842, 'price_change_24h': -8039.554638590213, 'price_change_percentage_24h': -8.75546, 'market_cap_change_24h': -159339683743.8938, 'market_cap_change_percentage_24h': -8.75837, 'circulating_supply': 19832031.0, 'total_supply': 19832031.0, 'max_supply': 21000000.0, 'ath': 108786, 'ath_change_percentage': -22.87821, 'ath_date': '2025-01-20T09:11:54.494Z', 'atl': 67.81, 'atl_change_percentage': 123625.97228, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-03-04T09:31:11.168Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 2090

[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400',
  'current_price': 83784,
  'market_cap': 1659945326387,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 1659945326387,
  'total_volume': 74109155624,
  'high_24h': 93668,
  'low_24h': 82842,
  'price_change_24h': -8039.554638590213,
  'price_change_percentage_24h': -8.75546,
  'market_cap_change_24h': -159339683743.8938,
  'market_cap_change_percentage_24h': -8.75837,
  'circulating_supply': 19832031.0,
  'total_supply': 19832031.0,
  'max_supply': 21000000.0,
  'ath': 108786,
  'ath_change_percentage': -22.87821,
  'ath_date': '2025-01-20T09:11:54.494Z',
  'atl': 67.81,
  'atl_change_percentage': 123625.97228,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2025-03-04T09:31:11.168Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://coin-images.coingecko.com/coins/images/

In [7]:
def analyze_data(data):
    df = pd.DataFrame(data)[["name", "symbol", "current_price", "market_cap", "total_volume", "price_change_percentage_24h"]]
    df.columns = ["Cryptocurrency Name", "Symbol", "Current Price (USD)", "Market Capitalization", "24h Trading Volume", "24h Price Change (%)"]
    
    top_5_by_market_cap = df.nlargest(5, "Market Capitalization")
    avg_price = df["Current Price (USD)"].mean()
    highest_24h_change = df.nlargest(1, "24h Price Change (%)")
    lowest_24h_change = df.nsmallest(1, "24h Price Change (%)")
    print(df, top_5_by_market_cap, avg_price, highest_24h_change, lowest_24h_change)
    return df, top_5_by_market_cap, avg_price, highest_24h_change, lowest_24h_change
    
data = fetch_crypto_data()  # Fetch live crypto data
df, top_5, avg_price, high_change, low_change = analyze_data(data)


[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 83784, 'market_cap': 1659945326387, 'market_cap_rank': 1, 'fully_diluted_valuation': 1659945326387, 'total_volume': 74109155624, 'high_24h': 93668, 'low_24h': 82842, 'price_change_24h': -8039.554638590213, 'price_change_percentage_24h': -8.75546, 'market_cap_change_24h': -159339683743.8938, 'market_cap_change_percentage_24h': -8.75837, 'circulating_supply': 19832031.0, 'total_supply': 19832031.0, 'max_supply': 21000000.0, 'ath': 108786, 'ath_change_percentage': -22.87821, 'ath_date': '2025-01-20T09:11:54.494Z', 'atl': 67.81, 'atl_change_percentage': 123625.97228, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-03-04T09:31:11.168Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?1696501628', 'current_price': 2090

In [8]:
# Function to update Excel sheet
def update_excel(df, top_5, avg_price, high_change, low_change):
    wb = Workbook()
    ws = wb.active
    ws.title = "Crypto Live Data"
    
    # Write main data
    for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)
    
    # Write analysis section
    ws.append([])
    ws.append(["Top 5 Cryptocurrencies by Market Cap"])
    for r in dataframe_to_rows(top_5, index=False, header=True):
        ws.append(r)
    
    ws.append([])
    ws.append(["Average Price of Top 50 Cryptocurrencies", avg_price])
    
    ws.append([])
    ws.append(["Highest 24h Price Change"])
    for r in dataframe_to_rows(high_change, index=False, header=True):
        ws.append(r)
    
    ws.append([])
    ws.append(["Lowest 24h Price Change"])
    for r in dataframe_to_rows(low_change, index=False, header=True):
        ws.append(r)
    
    wb.save("crypto_live_data.xlsx")
    print("Excel updated successfully!")
update_excel(df, top_5, avg_price, high_change, low_change)

Excel updated successfully!


In [9]:
# Main function to run the live updating script
def main():
    while True:
        print("Fetching latest crypto data...")
        data = fetch_crypto_data()
        df, top_5, avg_price, high_change, low_change = analyze_data(data)
        update_excel(df, top_5, avg_price, high_change, low_change)
        print("Waiting for next update (5 minutes)...")
        time.sleep(300)  # Wait for 5 minutes before next update

In [ ]:
# Run the script
if __name__ == "__main__":
    main()


Fetching latest crypto data...
[{'id': 'bitcoin', 'symbol': 'btc', 'name': 'Bitcoin', 'image': 'https://coin-images.coingecko.com/coins/images/1/large/bitcoin.png?1696501400', 'current_price': 83784, 'market_cap': 1659945326387, 'market_cap_rank': 1, 'fully_diluted_valuation': 1659945326387, 'total_volume': 74109155624, 'high_24h': 93668, 'low_24h': 82842, 'price_change_24h': -8039.554638590213, 'price_change_percentage_24h': -8.75546, 'market_cap_change_24h': -159339683743.8938, 'market_cap_change_percentage_24h': -8.75837, 'circulating_supply': 19832031.0, 'total_supply': 19832031.0, 'max_supply': 21000000.0, 'ath': 108786, 'ath_change_percentage': -22.87821, 'ath_date': '2025-01-20T09:11:54.494Z', 'atl': 67.81, 'atl_change_percentage': 123625.97228, 'atl_date': '2013-07-06T00:00:00.000Z', 'roi': None, 'last_updated': '2025-03-04T09:31:11.168Z'}, {'id': 'ethereum', 'symbol': 'eth', 'name': 'Ethereum', 'image': 'https://coin-images.coingecko.com/coins/images/279/large/ethereum.png?169